# Create Fake Data

This notebook demonstrates how to create fake data using the functions in `fake_data/fake_data_creator.py`. The data set matches the one created by the `make_demo_data()` in `fake_data/demo_helper.py`.

In [ ]:
import os

import matplotlib.pyplot as plt
from kbmod.fake_data.fake_data_creator import *
from kbmod.search import *

### Create a fake stack of images

In [ ]:
# Set the characteristics of the fake data.
img_width = 256
img_height = 256
num_times = 20

# Create the fake images
fake_times = create_fake_times(num_times, t0=57130.2)
ds = FakeDataSet(img_width, img_height, fake_times)

### Insert fake moving objects

This function creates a random moving object with a given flux that stays within the image for the entire time.  The trajectory is defined by starting pixels (x, y) and velocities (x_v, y_v) of pixels per day.

In [ ]:
trj = ds.insert_random_object(500)
print(f"x={trj.x}, y={trj.y}, xv={trj.vx}, yv={trj.vy}")

We can print the object's location at each time step.

In [ ]:
# Check the object was inserted correctly.
t0 = ds.stack.get_single_image(0).get_obstime()
for i in range(ds.stack.img_count()):
    ti = ds.stack.get_single_image(i).get_obstime()
    dt = ti - t0
    px = int(trj.x + dt * trj.vx + 0.5)
    py = int(trj.y + dt * trj.vy + 0.5)

    print(f"{i}: t={ti:.3f} at ({px}, {py})")

We can also create a known trajectory and insert an object using that trajectory. Here we create an object starting at x=50, y=10 and moving at 10 pixels per day in x and 0 pixels per day in y.

In [ ]:
trj2 = Trajectory(x=50, y=40, vx=10, vy=0, flux=500)
ds.insert_object(trj2)

We can plot the first and last images and we clearly see our two objects.

In [ ]:
plt.imshow(ds.stack.get_single_image(0).get_science().image)

In [ ]:
plt.imshow(ds.stack.get_single_image(num_times - 1).get_science().image)

### Save the fake image files

We can store the images in a `WorkUnit` and save that to disk (to be used in other notebooks).  In the `WorkUnit` we store a configuration with the minimal search parameters needed to retrieve the object we inserted. Specifically, the data has a known object moving at x_v=10 px/day and y_v = 0 px/day, so we set the search search velocities to [0, 20) and the search angles to [-0.5, 0.5). Note that these bounds may not find the other (random) trajectory we inserted.

In [ ]:
from kbmod.configuration import SearchConfiguration

settings = {
    # Override the search data to match the known object.
    "generator_config": {
        "name": "EclipticCenteredSearch",
        "angles": [-0.5, 0.5, 11],
        "velocities": [0.0, 20.0, 21],
        "angle_units": "radian",
        "force_ecliptic": 0.0,
    },
    # Loosen the other filtering parameters.
    "clip_negative": True,
    "sigmaG_lims": [15, 60],
    "mom_lims": [37.5, 37.5, 1.5, 1.0, 1.0],
    "peak_offset": [3.0, 3.0],
}
config = SearchConfiguration.from_dict(settings)

# We comment out this line so as not to overwrite the existing demo data.
# ds.save_fake_data_to_work_unit("../data/demo_data.fits", config=config)